In [1]:
import os.path as path
from pathlib import Path
import pandas as pd
import scipy.io
import gzip
import csv
import os
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

hr_path = "CytAssist_11mm_FFPE_Human_Ovarian_Carcinoma_tissue_image.tif"
dotpath = "spatial/tissue_positions.csv"

def truncated_dot_data(filepath):
    dotdf = pd.read_csv(filepath)
    filtered_dotdf = dotdf[dotdf['in_tissue'] != 0]
    filtered_dotdf.reset_index(drop=True, inplace=True)
    return filtered_dotdf

def make_10_crops(hr_path, tissue_positions_path):
    tissue_data = truncated_dot_data(tissue_positions_path)
    img = Image.open(hr_path)
    crops = []
    for i in range(10):
        x = tissue_data.at[tissue_data.index[i],'pxl_col_in_fullres']
        y = tissue_data.at[tissue_data.index[i],'pxl_row_in_fullres']
        new_image = img.crop((x - 110, y - 110, x + 110, y + 110))
        crops.append(new_image)
    
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
        
    return crops

def save_images(folder_name, images):
    output_folder = folder_name
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f"image_{i}.tif")  # Example: image_0.jpg, image_1.jpg, ...
        image.save(image_path)
        
def process_fbc_matrix():
    cd = Path(path.abspath(""))
    matrix_dir_path = os.path.join(cd, "filtered_feature_bc_matrix")
    mat_filtered = scipy.io.mmread(path.join(matrix_dir_path, "matrix.mtx.gz"))

    # list of transcript ids, e.g. 'ENSG00000187634'
    features_path = path.join(matrix_dir_path, "features.tsv.gz")
    feature_ids = [row[0]  for row  in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]

    # list of gene names, e.g. 'SAMD11'
    gene_names = [row[1]  for row  in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]

    # list of feature_types, e.g. 'Gene Expression'
    feature_types = [row[2]  for row  in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]

    # list of barcodes, e.g. 'AAACATACAAAACG-1'
    barcodes_path = os.path.join(matrix_dir_path, "barcodes.tsv.gz")
    barcodes = [row[0]  for row  in csv.reader(gzip.open(barcodes_path, mode="rt"), delimiter="\t")]
    
    fbc_matrix = pd.DataFrame.sparse.from_spmatrix(mat_filtered)
    fbc_matrix.columns = barcodes
    fbc_matrix.insert(loc=0, column="feature_id", value=feature_ids)
    fbc_matrix.insert(loc=1, column="gene", value=gene_names)
    fbc_matrix.insert(loc=2, column="feature_type", value=feature_types)
    return fbc_matrix

/Users/chengjunyuan/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
hr_path = "CytAssist_11mm_FFPE_Human_Ovarian_Carcinoma_tissue_image.tif"
dotpath = "spatial/tissue_positions.csv"

yok = make_10_crops(hr_path, dotpath)
save_images("yok", yok)

fbc_mat = process_fbc_matrix()
fbc_mat